In [ ]:
from baybe.core import BayBE
from baybe.parameters import Categorical, NumericDiscrete, GenericSubstance
from baybe.searchspace import SearchSpace
from baybe.strategies.bayesian import SequentialGreedyRecommender
from baybe.strategies.sampling import FPSRecommender
from baybe.strategies.strategy import Strategy
from baybe.targets import NumericalTarget, Objective

from pathlib import Path
import numpy as np
import json
from IPython.display import JSON

## Define the DOE problem

In [ ]:
# Parameters
parameters = [
    Categorical(
        name="Granularity",
        values=["coarse", "medium", "fine"],
        encoding="OHE",
    ),
    NumericDiscrete(
        name="Pressure[bar]",
        values=[1, 5, 10],
        tolerance=0.2,
    ),
    NumericDiscrete(
        name="Temperature[degree_C]",
        values=np.linspace(100, 200, 5).tolist(),  # temperature interval in steps of 25
    ),
    GenericSubstance(
        name="Solvent",
        data={
            'Solvent A':'COC', 
            'Solvent B':'CCCCC'},
        encoding="MORDRED",  # use cheminformatics encoding to inform the algorithm about chemistry
    )
]

# Targets
targets = [
    NumericalTarget(name="Yield", mode="MAX")
]

## Create the corresponding BayBE object

In [ ]:
baybe = BayBE(
    searchspace = SearchSpace.create(parameters  = parameters,
                                     constraints = None),
    objective   = Objective(mode    = 'SINGLE', 
                            targets = targets),
    strategy    = Strategy(recommender         = SequentialGreedyRecommender(),
                           initial_recommender = FPSRecommender(),)
)

## Get 5 recommendations for the next experiments

In [ ]:
recommendation_1 = baybe.recommend(5)
recommendation_1

## Simulate some random "Yield" measurement data

In [ ]:
measurements_1 =  recommendation_1.copy()
measurements_1.Yield = np.random.randint(0, 100, 5)
measurements_1

## Read in the simulated results and get the next 7 recommendations

In [ ]:
baybe.add_results(measurements_1)
recommendation_2 = baybe.recommend(7)
recommendation_2

## Store the current state of the project as JSON

In [ ]:
json_file = baybe.to_json()
JSON(json_file, expanded=False)

## Reload the project state from the JSON and verify that the next 7 recommendations are the same as above

In [ ]:
baybe_new = BayBE.from_json(json_file)
recommendation_2_reconstructed = baybe_new.recommend(7)
print(recommendation_2_reconstructed, end="\n\n")
print("Identical?", recommendation_2_reconstructed.equals(recommendation_2))

## Validate that the BayBE objects are identical

In [ ]:
print("Identical?", baybe == baybe_new)